In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

import numpy as np
import time
import random
import string
import pandas as pd
import copy
import os

os.chdir('/home/scott/Documents/EDx/MIT/Python/Spyder_scripts/')
WORDLIST_FILENAME = "words.txt"

#Read the file to a pandas object
data = pd.read_csv('firstLife.csv',sep=',',index_col=0).values

#convert the pandas object to a tensor
brain = tf.convert_to_tensor(data)


In [2]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    wordlist = tf.convert_to_tensor(wordlist)
    return wordlist

def chooseWord(wordlist):
    """
    wordlist (list): list of words (strings)

    Returns a word from wordlist at random
    """
    return np.random.choice(wordlist)

def isWordGuessed(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: boolean, True if all the letters of secretWord are in lettersGuessed;
      False otherwise
    '''
    for char in secretWord:
        if char not in lettersGuessed:
            return False
    
    return True  


def getGuessedWord(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters and underscores that represents
      what letters in secretWord have been guessed so far.
    '''
    
    word = str(secretWord)
    
    for char in word:
        if char not in lettersGuessed:
            word = word.replace(char,'_')
            
    return word
    


def getAvailableLetters(lettersGuessed):
    '''
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters that represents what letters have not
      yet been guessed.
    '''
    
    availableLetters = 'abcdefghijklmnopqrstuvwxyz'
    for char in lettersGuessed:
            position = availableLetters.index(char)
            availableLetters = availableLetters[0:position] + availableLetters[position+1::]
            
    return availableLetters

def Secrets():
    
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    # Construct pandas DataFrame called secrets that sorts all words in wordlist according to word length
    columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
    secrets = pd.DataFrame()
    
    for i in range(1,max(len(word) for word in wordlist)+1):
        dat = []
        for word in wordlist:
            if len(word) == i:
                dat.append(word)
        dat = pd.Series(dat)
        secrets = pd.concat([secrets,dat],axis=1, join='outer', join_axes=None, ignore_index=False)
    secrets.columns = columns
    
    return secrets

In [3]:
print(tf.test.is_gpu_available())
print(brain.device.endswith('GPU:0'))





True
False


In [4]:
def tensorPlayerAI(brain,numberOfGames):
    
    start = time.time()
    wordlist = loadWords()
    n = 1
    gamesWon = 0
    letters = list(string.ascii_lowercase)
    neuron = tfe.Variable(brain.numpy(), name= "titans")
    rawLetters = string.ascii_lowercase
    secrets = Secrets()
    
    while n <= numberOfGames:
        
        secretWord = np.random.choice(wordlist).decode("utf-8")
        lettersGuessed = ''
        numberOfGuesses = 8
        col = str(len(secretWord))
        words = secrets[col].dropna()
        words = list(words)
        print('Welcome to the game Hangman!')
        print('I am thinking of a word that is ' + str(len(secretWord)) + ' letters long.')
        print('-------------')
        game = True
        
        while game == True:
            
            options = copy.deepcopy(words)
            choices = secrets[str(len(secretWord))]
            availableLetters = getAvailableLetters(lettersGuessed)
            print('You have ' + str(numberOfGuesses) + ' guesses left.')
            print('Available letters: ' + availableLetters)
            best = 0
            
            if len(availableLetters) < 26:
                for chance in words:
                    for j in range(0,len(chance)):
                        if word[j] != '_' and  word[j] != chance[j]:
                            options.remove(chance)
                            break     
                           
            for letter in availableLetters:
        
                maximum = 0
                for word in options:
                    for char in word:
                        if char == letter:
                            maximum += 1
                            
                if maximum > best:
                    best = maximum
                    choice = letter
                    letterGuessed = choice
                    
            lettersGuessed = lettersGuessed[0::] + letterGuessed
            word = getGuessedWord(secretWord, lettersGuessed)
            letter = lettersGuessed[-1]  
            
            if lettersGuessed[-1] not in secretWord:
                print("Oops! That letter is not in my word: " + word)
                print('-------------')
                numberOfGuesses -= 1
                   
            else:
                print('Good guess: ' + word)
                print('-------------')
                for i in range(0,26):
                    if letters[i] == letterGuessed:
                        AI = neuron.numpy()
                        AI[i,len(secretWord)-1] += 1
                        neuron = tfe.Variable(AI, name= "titans")
                        
            answer = isWordGuessed(secretWord,lettersGuessed)
        
            if numberOfGuesses == 0 or answer == True:
                game = False    
           
        if answer == True:
            print('Congratulations, you won!')
            gamesWon += 1
            n += 1
            
        else:
            print('Sorry, you ran out of guesses. The word was ' + secretWord)
            n += 1
            
    print('Player function won ' + str(gamesWon) + ' games.')
    print('Player function lost ' + str(numberOfGames - gamesWon) + ' games.')
    end = time.time()
    print('PlayerAI played ' + str(numberOfGames) + ' games in '+ str(round((end-start)/60,2)) + ' minutes')
    
    return [numberOfGames,brain]
        

In [6]:
numberOfGames = 10
[n,brain] = tensorPlayerAI(brain,numberOfGames)

Loading word list from file...
   55909 words loaded.
Welcome to the game Hangman!
I am thinking of a word that is 8 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _e______
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _e______
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqstuvwxyz
Oops! That letter is not in my word: _e______
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: _e___i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqtuvwxyz
Good guess: _e___in_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqtuvwxyz
Good guess: _e___ing
-------------
You have 6 guesses left.
Available letters: abcdfhjklmopqtuvwxyz
Good guess: _edd_ing
-------------
You have 6 guesses left.
Available letters: abcfhjklmopqtuvwxyz
Good guess: _eddling

-------------
You have 2 guesses left.
Available letters: cdgkmnopqrtuvwxyz
Oops! That letter is not in my word: _ail
-------------
You have 1 guesses left.
Available letters: cdgknopqrtuvwxyz
Oops! That letter is not in my word: _ail
-------------
Sorry, you ran out of guesses. The word was pail
Welcome to the game Hangman!
I am thinking of a word that is 8 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ____i___
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Oops! That letter is not in my word: ____i___
-------------
You have 5 guesses left.
Available letters: abcdfghjklmnopqruvwxyz
Good guess: ____i__n
-------------
You have 5 guess